# Example to make a Rotating Globe

Let's have some fun by trying to make a rotating globe, display some locations fading in and out with time, maybe even add the day/night shadows for more fun and ... Well, I just hope that it looks somewhat nice at the end when using python routines only (otherwise I might need to do some more advanced image magic or even worse 3D modelling). 

## Globe Animation (with cartopy based on Natural Earth data)

As a baseline, I've used the following answer on stackoverflow to produce a quick animation: 
https://stackoverflow.com/questions/51801109/animate-a-point-between-two-points-along-with-rotating-earth

In [ ]:
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from shapely.geometry import Polygon

import numpy as np

import matplotlib.animation as animation
import matplotlib.pyplot as plt
%matplotlib notebook

plt.figure(figsize=(4,4))

def decorate_axes(ax):
    ax.set_global()
    ax.add_feature(cfeature.OCEAN)
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle=':',linewidth=0.3, alpha=.5)

    
def animate(i):
    lon = i

    ax = plt.gca()
    ax.remove()

    ax = plt.axes([0, 0, 1, 1], projection=ccrs.Orthographic(
        central_latitude=0, central_longitude=lon))
    decorate_axes(ax)


ani = animation.FuncAnimation(
    plt.gcf(), animate,
    frames=np.linspace(360, 0, 60),
    interval=125, repeat=False)

plt.show()

## Adding Geographical Locations

We want to show a few locations on the map. So we use the geo-locations of the participants of the International Cosmic Day (https://icd.desy.de/). A day, where students, teachers and scientists from all around the globe come together to learn about cosmic rays. 

In [ ]:
#Pink CMYK 0/100/0/0 (oder von mir ausgelesen RGB 222/0/123)  == eb588d
#Blau CMYK 89/0/8/0 (oder von mir ausgelesen RGB 0/165/219) == 349ac0 == 3db5e3

icdpink='#eb588d'
icdblue='#3db5e3'

In [ ]:
import geopandas as gpd

# read in geo location from google maps export
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

fp = "/home/prokoph/DESY/outreach/2021_ICD/Participants2020.kml"
df = gpd.read_file(fp, driver='KML')

fp = "/home/prokoph/DESY/outreach/2021_ICD/HESS.kml"
dfhess = gpd.read_file(fp, driver='KML')

fp2012 = "/home/prokoph/DESY/outreach/2021_ICD/2012.kml"
df2012 = gpd.read_file(fp2012, driver='KML')
fp2013 = "/home/prokoph/DESY/outreach/2021_ICD/International Cosmic Day 2013.kml"
df2013 = gpd.read_file(fp2013, driver='KML')
fp2014 = "/home/prokoph/DESY/outreach/2021_ICD/International Cosmic Day 2014.kml"
df2014 = gpd.read_file(fp2014, driver='KML')
fp2015 = "/home/prokoph/DESY/outreach/2021_ICD/2015.kml"
df2015 = gpd.read_file(fp2015, driver='KML')
fp2016 = "/home/prokoph/DESY/outreach/2021_ICD/International Cosmic Day 2016.kml"
df2016 = gpd.read_file(fp2016, driver='KML')
fp2017 = "/home/prokoph/DESY/outreach/2021_ICD/International Cosmic Day 2017.kml"
df2017 = gpd.read_file(fp2017, driver='KML')
fp2018 = "/home/prokoph/DESY/outreach/2021_ICD/International Cosmic Day 2018.kml"
df2018 = gpd.read_file(fp2018, driver='KML')
fp2019 = "/home/prokoph/DESY/outreach/2021_ICD/International Cosmic Day 2019.kml"
df2019 = gpd.read_file(fp2019, driver='KML')
fp2020 = "/home/prokoph/DESY/outreach/2021_ICD/International Cosmic Day 2020.kml"
df2020 = gpd.read_file(fp2020, driver='KML')
fp2021 = "/home/prokoph/DESY/outreach/2021_ICD/International Cosmic Day 2021.kml"
df2021 = gpd.read_file(fp2021, driver='KML')
fp2022 = "/home/prokoph/DESY/outreach/2022_ICD/Participating Institutions.kml"
df2022 = gpd.read_file(fp2022, driver='KML')
fp2023 = "/home/prokoph/DESY/outreach/2023_ICD/cities and institutes.kml"
df2023 = gpd.read_file(fp2023, driver='KML')

In [ ]:
import geopandas as gpd

import cartopy
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
from cartopy.feature import ShapelyFeature

# Read shape file  - get country borders
resolution = '110m' #'10m'
category = 'cultural'
name = 'admin_0_countries'

# read the natural earth shapefile using geopandas
shpfilename = shpreader.natural_earth(resolution, category, name)
world = gpd.read_file(shpfilename)

world.plot()

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=cartopy.crs.Mollweide())
plt.axis('off')
world.plot(ax=ax, edgecolor="white", facecolor='lightgray', lw=0.25)
# df2012.plot(ax=ax, alpha=0.3,markersize=3,facecolor='blue')
# df2013.plot(ax=ax, alpha=0.3,markersize=3,facecolor='blue')
# df2014.plot(ax=ax, alpha=0.3,markersize=3,facecolor='blue')
# df2015.plot(ax=ax, alpha=0.3,markersize=3,facecolor='blue')
# df2016.plot(ax=ax, alpha=0.3,markersize=3,facecolor='blue')
# df2017.plot(ax=ax, alpha=0.3,markersize=3,facecolor='blue')
# df2018.plot(ax=ax, alpha=0.3,markersize=3,facecolor='blue')
# df2019.plot(ax=ax, alpha=0.3,markersize=3,facecolor='blue')
# df2020.plot(ax=ax, alpha=0.3,markersize=3,facecolor='blue')
# dfhess.plot(ax=ax, alpha=0.3,markersize=3,facecolor='blue')
# df2021.plot(ax=ax, alpha=0.3,markersize=3,facecolor='blue')
# df2022.plot(ax=ax, alpha=0.3,markersize=3,facecolor='blue')
df2012.plot(ax=ax,markersize=3,facecolor='blue')
df2013.plot(ax=ax,markersize=3,facecolor='blue')
df2014.plot(ax=ax,markersize=3,facecolor='blue')
df2015.plot(ax=ax,markersize=3,facecolor='blue')
df2016.plot(ax=ax,markersize=3,facecolor='blue')
df2017.plot(ax=ax,markersize=3,facecolor='blue')
df2018.plot(ax=ax,markersize=3,facecolor='blue')
df2019.plot(ax=ax,markersize=3,facecolor='blue')
df2020.plot(ax=ax,markersize=3,facecolor='blue')
dfhess.plot(ax=ax,markersize=3,facecolor='blue')
df2021.plot(ax=ax,markersize=3,facecolor='blue')
df2022.plot(ax=ax,markersize=3,alpha=0.7,facecolor='none',edgecolor='red',linewidth=1)

# plt.savefig('../world_2022highlighted.png',dpi=300)

In [ ]:
# Read shape file  - get country borders
resolution = '50m' #'10m'
category = 'cultural'
name = 'admin_0_countries'

# read the natural earth shapefile using geopandas
shpfilename = shpreader.natural_earth(resolution, category, name)
world_hires = gpd.read_file(shpfilename)

europe=world_hires[world_hires['CONTINENT'] == 'Europe']

#Remove Russia and Iceland from map of Europe
europe=europe[(europe['ADMIN']!='Russia') & (europe['ADMIN']!='Iceland')]
# europe.plot()

# Create a custom polygon
polygon = Polygon([(-15,35), (40,35), (40,75),(-15,75)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[polygon], crs=world.crs)

# fig,ax=plt.subplots()
# ax=europe.plot(ax=ax)
# poly_gdf.plot(edgecolor='red',ax=ax, alpha=0.1)
# plt.show()

#Clip polygon from the map of Europe
europe=gpd.clip(europe, polygon) 
europe.plot()#europe


In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=cartopy.crs.Mollweide())
plt.axis('off')
europe.plot(ax=ax, edgecolor="white", facecolor='lightgray', lw=0.25)

df12=gpd.clip(df2012, polygon) 
df12.plot(ax=ax,markersize=20,facecolor='blue')
df13=gpd.clip(df2013, polygon) 
df13.plot(ax=ax,markersize=20,facecolor='blue')
df14=gpd.clip(df2014, polygon) 
df14.plot(ax=ax,markersize=20,facecolor='blue')
df15=gpd.clip(df2015, polygon) 
df15.plot(ax=ax,markersize=20,facecolor='blue')
df16=gpd.clip(df2017, polygon) 
df16.plot(ax=ax,markersize=20,facecolor='blue')
df17=gpd.clip(df2017, polygon) 
df17.plot(ax=ax,markersize=20,facecolor='blue')
df18=gpd.clip(df2018, polygon) 
df18.plot(ax=ax,markersize=20,facecolor='blue')
df19=gpd.clip(df2019, polygon) 
df19.plot(ax=ax,markersize=20,facecolor='blue')
df20=gpd.clip(df2020, polygon) 
df20.plot(ax=ax,markersize=20,facecolor='blue')
df21=gpd.clip(df2021, polygon) 
df21.plot(ax=ax,markersize=20,facecolor='blue')
df22=gpd.clip(df2022, polygon) 
df22.plot(ax=ax,markersize=22,facecolor='none',edgecolor='red',linewidth=1)

# plt.savefig('../europe_2022highlighted.png',dpi=300)

In [ ]:
def decorate_axes(ax):
    ax.set_global()
    ax.add_feature(cfeature.OCEAN,facecolor=icdblue)
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle=':',linewidth=0.3, alpha=.5)

    
def animate(i):
    lon = i

    ax = plt.gca()
    ax.remove()

    # Define the CartoPy CRS object.
    globe_proj = ccrs.Orthographic(central_latitude=23, central_longitude=lon)

    # convert geolocations into a `proj4` string/dict compatible with GeoPandas
    crs_proj4 = globe_proj.proj4_init
    df_orto = df.to_crs(crs_proj4)

    ax = plt.axes([0, 0, 1, 1], projection=globe_proj)
    df_orto.plot(ax=ax,edgecolor=icdpink,facecolor=icdpink, alpha=0.5, marker='o')
    
    decorate_axes(ax)


In [ ]:
plt.figure(figsize=(7,7))

def decorate_axes(ax):
    ax.set_global()
    ax.add_feature(cfeature.OCEAN)
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle=':',linewidth=0.3, alpha=.5)

# Define the CartoPy CRS object. This one will change during the animation. So let's keep it more general here.
globe_proj = ccrs.Orthographic(central_latitude=23, central_longitude=-20)

# Now, we need to convert our GeoPandas data points into the same crs as the cartopy globe projection. 
# So we convert cartopy crs into a `proj4` string/dict compatible with GeoPandas and apply it to the data.
crs_proj4 = globe_proj.proj4_init
df_orto = df2023.to_crs(crs_proj4)

ax = plt.axes([0, 0, 1, 1], projection=globe_proj)
df_orto.plot(ax=ax,edgecolor='None',facecolor='red', alpha=0.5, marker='o')
decorate_axes(ax)


In [ ]:
plt.figure(figsize=(10,10))

def decorate_axes(ax):
    ax.set_global()
    ax.add_feature(cfeature.OCEAN,facecolor=icdblue)
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle=':',linewidth=0.3, alpha=.5)

    
def animate(i):
    lon = i

    ax = plt.gca()
    ax.remove()

    # Define the CartoPy CRS object.
    globe_proj = ccrs.Orthographic(central_latitude=23, central_longitude=lon)

    # convert geolocations into a `proj4` string/dict compatible with GeoPandas
    crs_proj4 = globe_proj.proj4_init
    df_orto = df2023.to_crs(crs_proj4)

    ax = plt.axes([0, 0, 1, 1], projection=globe_proj)
    df_orto.plot(ax=ax,edgecolor=None,facecolor=icdpink, alpha=0.8, marker='o')
    
    decorate_axes(ax)


ani = animation.FuncAnimation(
    plt.gcf(), animate,
    frames=np.linspace(180,-180, 60),
    interval=125, repeat=False)

# turn off axis spines
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
ax.set_frame_on(False)

# set figure background opacity (alpha) to 0
fig.patch.set_alpha(0.0)


plt.show()
ani.save('../globe_2023.gif', writer='imagemagick', dpi=plt.gcf().dpi )
